In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.3 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [5]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [6]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


In [8]:
# 🧩 Task 1 — Modify the Oracle
# Create your own balanced oracle function that flips the ancilla for half of all possible inputs.
# Hint: You can use cx and ccx gates creatively.

from qiskit import QuantumCircuit

def oracle_custom_balanced(qc, inputs, ancilla):
    """
    Example of a custom balanced oracle that flips the ancilla
    for half of all possible inputs (not just parity).
    """
    # Example: Flip ancilla when x0 AND NOT x1
    qc.cx(inputs[0], ancilla)
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.x(inputs[1])       # Negate one qubit to create new pattern
    qc.ccx(inputs[0], inputs[1], ancilla)
    qc.x(inputs[1])

# Test the oracle structure
qc = QuantumCircuit(3 + 1)
oracle_custom_balanced(qc, [0,1,2], 3)
qc.draw('text')


q_0: ──■────■─────────■───────
       │    │  ┌───┐  │  ┌───┐
q_1: ──┼────■──┤ X ├──■──┤ X ├
       │    │  └───┘  │  └───┘
q_2: ──┼────┼─────────┼───────
     ┌─┴─┐┌─┴─┐     ┌─┴─┐     
q_3: ┤ X ├┤ X ├─────┤ X ├─────
     └───┘└───┘     └───┘

In [9]:
# ⚙️ Task 2 — Change the Number of Input Qubits
# Run the Deutsch–Jozsa algorithm with different numbers of input qubits
# (e.g., n = 2, 4, 5) and observe how the circuit depth and output pattern change.

for n in [2, 4, 5]:
    print(f"\n=== Running Deutsch–Jozsa with n = {n} qubits ===")
    qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_parity, list(range(n)), n)
    print("Circuit depth:", qc_balanced.depth())
    run_dj(qc_balanced)



=== Running Deutsch–Jozsa with n = 2 qubits ===
Circuit depth: 6
Measurement counts: {'11': 1024}
✅ Function is BALANCED

=== Running Deutsch–Jozsa with n = 4 qubits ===
Circuit depth: 8
Measurement counts: {'1111': 1024}
✅ Function is BALANCED

=== Running Deutsch–Jozsa with n = 5 qubits ===
Circuit depth: 9
Measurement counts: {'11111': 1024}
✅ Function is BALANCED


In [11]:
# 📈 Task 3 — Add Noise Simulation (Fixed)
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer import AerSimulator
from qiskit import transpile

# Create a noise model
noise_model = NoiseModel()

# 1-qubit depolarizing error for single-qubit gates (e.g., H)
error_1 = depolarizing_error(0.02, 1)

# 2-qubit depolarizing error for CNOT gates
error_2 = depolarizing_error(0.05, 2)

# Add to the noise model
noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

# Prepare and run the circuit
n = 3
qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_parity, list(range(n)), n)
simulator_noisy = AerSimulator(noise_model=noise_model)
tqc = transpile(qc_balanced, simulator_noisy)
result = simulator_noisy.run(tqc, shots=1024).result()
counts_noisy = result.get_counts()

print("Noisy Measurement Counts:", counts_noisy)
plot_histogram(counts_noisy)
plt.show()


Noisy Measurement Counts: {'001': 33, '110': 32, '010': 1, '000': 41, '101': 30, '011': 59, '111': 828}


In [13]:
!pip install qiskit-ibm-runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00


In [14]:
# 💡 Task 4 — Run on IBM Quantum Device
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit import transpile

# Load your IBM Quantum account (make sure you’ve saved your token once using QiskitRuntimeService.save_account())
service = QiskitRuntimeService()

# Choose backend (you can list available ones with `service.backends()`)
backend = service.backend("ibm_oslo")  # Example device; you can choose another

# Prepare Deutsch–Jozsa circuit
n = 3
qc_balanced = deutsch_jozsa_circuit(n, oracle_balanced_parity, list(range(n)), n)
tqc = transpile(qc_balanced, backend)

# Create and run SamplerV2 job
sampler = SamplerV2(backend)
job = sampler.run([tqc])
result = job.result()

# Get and visualize results
print("Measurement results from real IBM Quantum device:")
print(result)


AccountNotFoundError: "Unable to find account. Please make sure an account with the channel name 'ibm_quantum_platform' is saved."

In [16]:
# 🧮 Task 5 — Circuit Analysis
# Print the unitary of your oracle and explain how it implements f(x)

qc = QuantumCircuit(3 + 1)
oracle_balanced_parity(qc, [0,1,2], 3)

# Convert to gate and print its definition
oracle_gate = qc.to_gate(label="Oracle")
print(oracle_gate.definition)

# Visualization
qc.draw('text')


                    
q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘


q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘